In [56]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns

iris = load_iris()
X = iris.data       
y = iris.target     

df = pd.DataFrame(X, columns=iris.feature_names)

# X.shape
X.shape[0]

# Nombre de centroïdes
K = 4

#### 1.  On place k  (centroïdes) au hasard parmi les données

In [57]:
def create_centroids(X, K):
    # Nombre de centroïde
    k = 4
    #  k indices différents et aléatoires parmi les lignes de X, sans répétition.
    indices = np.random.choice(X.shape[0], k, replace=False)
    # tableau (k, 4) contenant k centroïdes initiaux choisis parmi les points du dataset.
    initial_centroids = X[indices]

    return initial_centroids


In [58]:
# TEST DE LA FONCTION
# initial_centroids = create_centroids(X, K)
# initial_centroids

#### 2. Chaque point du dataset est affecté au centroïd le plus proche 
##### fonction `assign_to_clusters`


- `np.newaxis` →      ajoute une dimension

- `X[:, np.newaxis]` →  transforme X de (150, 4) en (150, 1, 4)

- `centroids.shape` →  (k, 4)

- `X[:, np.newaxis - centroids]` →  (150, k, 4)

- `np.linalg.norm(..., axis=2)` →  calcule la distance euclidienne → forme finale : (150, k)

- `np.argmin(..., axis=1)` →  donne pour chaque ligne, l’indice du centroïde le plus proche

In [59]:
def assign_to_clusters(X, initial_centroids):
    # Calcul des distances entre chaque point et chaque centroïde
    distances = np.linalg.norm(X[:, np.newaxis] - initial_centroids, axis=2)
    
    # Affectation : indice du centroïde le plus proche
    cluster_labels = np.argmin(distances, axis=1)
    
    return cluster_labels    

In [60]:
# TEST DE LA FONCTION
# cluster_labels = assign_to_clusters(X, initial_centroids)
# cluster_labels

#### 3. Le centroïd est déplacé au milieu du cluster (la moyenne des points)
##### fonction `update_centroids`

- `labels == i` sélectionne tous les points du cluster 𝑖

- `.mean(axis=0)` calcule la moyenne colonne par colonne

- `np.zeros((k, 4))` initialise la nouvelle matrice de centroïdes

In [61]:
def update_centroids(X, cluster_labels, K):
    new_centroids = np.zeros((K, X.shape[1]))
    for i in range(K):
        cluster_points = X[cluster_labels == i]
        if len(cluster_points) > 0:
            new_centroids[i] = cluster_points.mean(axis=0)
    return new_centroids

In [62]:
# TEST DE LA FONCTION
# new_centroids = update_centroids(X, cluster_labels, K)
# new_centroids

#### 4.  on recommence jusqu'à ce que les centroïds convergent vers le point d'équilibre

In [63]:
centroids = create_centroids(X, K)

for iteration in range(10):  # ou jusqu'à convergence
    cluster_labels = assign_to_clusters(X, centroids)
    new_centroids = update_centroids(X, cluster_labels, K)
    
    if np.allclose(new_centroids, centroids, atol=1e-4):
        print(f"Convergence à l’itération {iteration}")
        break
    
    centroids = new_centroids


Convergence à l’itération 3


In [64]:
print("Centroids finaux :\n", centroids,"\n")
print("Exemples de labels assignés :", cluster_labels[:10])

Centroids finaux :
 [[5.88360656 2.74098361 4.38852459 1.43442623]
 [4.81818182 3.23636364 1.43333333 0.23030303]
 [6.85384615 3.07692308 5.71538462 2.05384615]
 [5.37058824 3.8        1.51764706 0.27647059]] 

Exemples de labels assignés : [1 1 1 1 1 3 1 1 1 1]
